In [1]:
# conda activate torch_gpu
import os
import pandas as pd
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import GPT2LMHeadModel

/mnt/data/tools/miniconda3/envs/torch_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/data/tools/miniconda3/envs/torch_gpu/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M

In [3]:
# model_hf

In [4]:
# This class is combination of 2 classes we generated in gpt_dev_aj i.e. class Head and MultiHeadAttention
# class CausalSelfAttention(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         # key, query and value for all the heads in a batch
#         self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
#         # output layer
#         self.c_proj = nn.Linear(config.n_embd, config.n_embd)
#         self.n_head = config.n_head
#         self.n_embd = config.n_embd
#         self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1,1,config.block_size, config.block_size))

#     def forward(self, x):
#         B,T,C = x.shape
#         qkv = self.c_attn(x)
#         q,k,v = qkv.split(self.n_embd, dim=2)
#         k = k.view(B, T, self.n_head, C // self.n_head).transpose(1,2)
#         q = q.view(B, T, self.n_head, C // self.n_head).transpose(1,2)
#         v = v.view(B, T, self.n_head, C // self.n_head).transpose(1,2)
#         y = F.scaled_dot_product_attention(q,k,v, is_causal = True)
#         y = y.transpose(1,2).contiguous().view(B,T,C)
#         y = self.c_proj(y)
#         return y 


In [2]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

In [3]:
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1 

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x) 
        x = self.c_proj(x)
        return x


In [4]:
# No need to approximate it but we are replicating GPT2 so we will keep this
# GELU is similar to RELU instead it does not have any dead neuron as in RELU if x < 0 so y = 0

# class MLP(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(config.n_embd, 4 * config.n_embd),
#             nn.GELU(approximate = "tanh"),
#             nn.Linear(4 * config.n_embd, config.n_embd),
#             nn.Dropout(config.dropout),
#         )

#     def forward(self, x):
#         return(self.net(x))

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        # head_size = self.n_embd // self.n_head
        # self.sa_heads = MultiHeadAttention(self.n_head, head_size)
        # self.fwd = FeedForwardLayer(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.mlp = MLP(config)
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x


In [5]:
class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


In [6]:
@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

In [7]:
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.transformer.wte.weight = self.lm_head.weight

        # Trying to make it more like GPT2 so initialization has set the distribution as 0.2 std dev
        self.apply(self._init_weights)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean = 0.0, std = std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = std)

    def forward(self, idx, targets = None):
        B, T = idx.shape
        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(torch.arange(T, dtype=torch.long, device = idx.device))
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            # print(k)
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

# Using GPT2 pretrained Weights

In [ ]:
## This will pass all the weights and bias from gpt2 to our model
model = GPT.from_pretrained('gpt2')
print("Yayy it worked!!!")

loading weights from pretrained gpt: gpt2
Yayy it worked!!!


In [ ]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [37]:
### Now we will evaluate our model
model.eval()
model.to('cuda')

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [100]:
max_return_sequences = 5 # Batch or number of rows
max_length = 30

import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello I love transformers")
tokens = torch.tensor(tokens, dtype = torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(max_return_sequences, 1) # (5,8)
x = tokens.to('cuda')
print(tokens)

tensor([[15496,   314,  1842,  6121,   364],
        [15496,   314,  1842,  6121,   364],
        [15496,   314,  1842,  6121,   364],
        [15496,   314,  1842,  6121,   364],
        [15496,   314,  1842,  6121,   364]])


In [101]:
torch.manual_seed(20002)
torch.cuda.manual_seed(20002)

while x.size(1) < max_length:
    with torch.no_grad():
        logits = model(x) # (B,T,vocab_size)
        print(logits[0], logits.shape)
        ## Taking the logits at the last location
        logits = logits[:,-1,:] # (B, vocab_size)
        print(logits[0], logits.shape)
        # get probabilities
        prob = F.softmax(logits, dim = -1)
        # top_k probabilities sampling for top 50 probabilities
        topk_prob, topk_indices = torch.topk(prob, 50, dim= -1)
        print(topk_prob, topk_indices)
        ## Finding the best next probabible token but randomly choosing
        ix_next = torch.multinomial(topk_prob, num_samples=1) # This provides the index
        # print(ix_next)
        # gather the corresponding col
        # Provide the actual token ID
        xcol = torch.gather(topk_indices, -1, ix_next)
        # print(xcol)
        # append to the sequences
        # Adding to the x tokens
        x = torch.cat((x, xcol), dim=1)
        # print(ix)

tensor([[ -35.2362,  -35.3266,  -38.9753,  ...,  -44.4645,  -43.9974,
          -36.4580],
        [-149.1651, -149.5904, -153.6234,  ..., -159.2461, -152.7463,
         -151.8448],
        [ -86.0484,  -87.9332,  -92.1532,  ...,  -93.8557,  -94.7155,
          -89.1611],
        [ -76.5149,  -79.7025,  -82.4027,  ...,  -88.3354,  -89.0814,
          -81.1800],
        [ -89.0248,  -93.8441,  -97.9009,  ..., -105.7052, -105.5819,
          -94.6414]], device='cuda:0') torch.Size([5, 5, 50257])
tensor([ -89.0248,  -93.8441,  -97.9009,  ..., -105.7052, -105.5819,
         -94.6414], device='cuda:0') torch.Size([5, 50257])
tensor([[0.2526, 0.1887, 0.1331, 0.1060, 0.0290, 0.0256, 0.0196, 0.0104, 0.0082,
         0.0070, 0.0070, 0.0070, 0.0061, 0.0061, 0.0060, 0.0052, 0.0051, 0.0050,
         0.0048, 0.0046, 0.0045, 0.0036, 0.0036, 0.0036, 0.0034, 0.0033, 0.0030,
         0.0030, 0.0028, 0.0027, 0.0026, 0.0023, 0.0022, 0.0021, 0.0020, 0.0018,
         0.0018, 0.0016, 0.0015, 0.0015, 0.0014,

In [102]:
print(logits[0], logits.shape) ### This is the logits for the last layer
print(prob[0], prob.shape) ## Probability of the next token
print(topk_prob[0],topk_prob.shape) # Probability of the 50 best tokens
print(topk_indices[0],topk_indices.shape) # Probability of the 50 best tokens indices
print(ix_next) ## This is randomly choosen value index for next from the top 50 probability
print(xcol)
print(x)

tensor([-115.5523, -119.1328, -126.3796,  ..., -129.3909, -129.5217,
        -119.3004], device='cuda:0') torch.Size([5, 50257])
tensor([2.6529e-03, 7.3916e-05, 5.2663e-08,  ..., 2.5924e-09, 2.2744e-09,
        6.2511e-05], device='cuda:0') torch.Size([5, 50257])
tensor([0.4332, 0.0740, 0.0649, 0.0638, 0.0517, 0.0431, 0.0308, 0.0249, 0.0191,
        0.0132, 0.0116, 0.0113, 0.0090, 0.0090, 0.0069, 0.0061, 0.0058, 0.0050,
        0.0049, 0.0040, 0.0038, 0.0037, 0.0037, 0.0035, 0.0033, 0.0031, 0.0031,
        0.0029, 0.0028, 0.0027, 0.0027, 0.0024, 0.0020, 0.0019, 0.0019, 0.0019,
        0.0018, 0.0016, 0.0016, 0.0014, 0.0014, 0.0013, 0.0013, 0.0012, 0.0012,
        0.0011, 0.0011, 0.0010, 0.0009, 0.0009], device='cuda:0') torch.Size([5, 50])
tensor([ 503,   11,  326,   13,  290,  314,  287,  475,  319,  284,  588,  329,
         345,  994,  612,  523,  996,  543, 1088, 1165,  351,  379, 1695,  422,
         826,  783, 2691, 1541,  532,  286,    0,  355,  780,  810, 2406,   25,
         5

In [104]:
for i in range(max_return_sequences):
    token = x[i,:max_length].tolist()
    # print(token)
    text = enc.decode(token)
    print(">", text)

> Hello I love transformers I think it's great and in the sense I can actually do a lot of things. There are so many great ones in
> Hello I love transformers, and I use them often, but because they're so nice and useful for a few weeks now I have had to make
> Hello I love transformers, and I did not have any problems getting them to work.

I hope this makes it easier to find your source
> Hello I love transformers

RJI: I feel a lot better if I had a little more information on them. And so far I
> Hello I love transformers and the animation I've done with mine. I'll do a lot more with you and you don't have too many tricks


In [105]:
### I found something interesting suppose if my text is same but with the ! mark. Nothing makes any sense
max_return_sequences = 5 # Batch or number of rows
max_length = 30

import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello I love transformers!")
tokens = torch.tensor(tokens, dtype = torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(max_return_sequences, 1) # (5,8)
x = tokens.to('cuda')
print(tokens)

tensor([[15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0]])


In [106]:
torch.manual_seed(20002)
torch.cuda.manual_seed(20002)

while x.size(1) < max_length:
    with torch.no_grad():
        logits = model(x) # (B,T,vocab_size)
        print(logits[0], logits.shape)
        ## Taking the logits at the last location
        logits = logits[:,-1,:] # (B, vocab_size)
        print(logits[0], logits.shape)
        # get probabilities
        prob = F.softmax(logits, dim = -1)
        # top_k probabilities sampling for top 50 probabilities
        topk_prob, topk_indices = torch.topk(prob, 50, dim= -1)
        print(topk_prob, topk_indices)
        ## Finding the best next probabible token but randomly choosing
        ix_next = torch.multinomial(topk_prob, num_samples=1) # This provides the index
        # print(ix_next)
        # gather the corresponding col
        # Provide the actual token ID
        xcol = torch.gather(topk_indices, -1, ix_next)
        # print(xcol)
        # append to the sequences
        # Adding to the x tokens
        x = torch.cat((x, xcol), dim=1)
        # print(ix)

tensor([[ -35.2362,  -35.3266,  -38.9753,  ...,  -44.4645,  -43.9974,
          -36.4580],
        [-149.1651, -149.5904, -153.6234,  ..., -159.2461, -152.7463,
         -151.8448],
        [ -86.0484,  -87.9332,  -92.1532,  ...,  -93.8557,  -94.7155,
          -89.1611],
        [ -76.5149,  -79.7025,  -82.4027,  ...,  -88.3354,  -89.0814,
          -81.1800],
        [ -89.0248,  -93.8441,  -97.9009,  ..., -105.7052, -105.5819,
          -94.6414],
        [-103.7818, -106.4338, -106.1750,  ..., -115.6992, -115.0282,
         -100.7213]], device='cuda:0') torch.Size([5, 6, 50257])
tensor([-103.7818, -106.4338, -106.1750,  ..., -115.6992, -115.0282,
        -100.7213], device='cuda:0') torch.Size([5, 50257])
tensor([[0.2344, 0.1487, 0.0737, 0.0302, 0.0230, 0.0214, 0.0145, 0.0135, 0.0111,
         0.0106, 0.0105, 0.0103, 0.0102, 0.0079, 0.0077, 0.0066, 0.0066, 0.0053,
         0.0052, 0.0052, 0.0049, 0.0048, 0.0045, 0.0044, 0.0042, 0.0039, 0.0036,
         0.0035, 0.0033, 0.0032, 0.003

In [107]:
for i in range(max_return_sequences):
    token = x[i,:max_length].tolist()
    # print(token)
    text = enc.decode(token)
    print(">", text)

> Hello I love transformers! Thank you so much! Thank you for helping out the team.

" I love all of your products, and
> Hello I love transformers!

I really like the look of most things in my cube, especially if it is built for the real thing.
> Hello I love transformers! So I took my first one, and now I'm back to my first one.

All I need now to
> Hello I love transformers! My wife and I spent a lot of the summer working on their home. I took out my camera and was shooting some
> Hello I love transformers! They make me look like a little doll!

This is a really fun product but it took me a very long


# GPT2 from Scratch

In [8]:
## Now for training you surely need to have GPU or cuda
# device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends,"mps") and torch.backends.mps.is_available():
    device = "mps"

print("device name",device)

device name cuda


In [8]:
## This will pass all the weights and bias from gpt2 to our model
model = GPT(GPTConfig())
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [9]:
max_return_sequences = 5 # Batch or number of rows
max_length = 30

import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello I love transformers!")
tokens = torch.tensor(tokens, dtype = torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(max_return_sequences, 1) # (5,8)
x = tokens.to(device)
print(tokens)

tensor([[15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0],
        [15496,   314,  1842,  6121,   364,     0]])


In [114]:
torch.manual_seed(20002)
torch.cuda.manual_seed(20002)

while x.size(1) < max_length:
    with torch.no_grad():
        logits = model(x) # (B,T,vocab_size)
        print(logits[0], logits.shape)
        ## Taking the logits at the last location
        logits = logits[:,-1,:] # (B, vocab_size)
        print(logits[0], logits.shape)
        # get probabilities
        prob = F.softmax(logits, dim = -1)
        # top_k probabilities sampling for top 50 probabilities
        topk_prob, topk_indices = torch.topk(prob, 50, dim= -1)
        print(topk_prob, topk_indices)
        ## Finding the best next probabible token but randomly choosing
        ix_next = torch.multinomial(topk_prob, num_samples=1) # This provides the index
        # print(ix_next)
        # gather the corresponding col
        # Provide the actual token ID
        xcol = torch.gather(topk_indices, -1, ix_next)
        # print(xcol)
        # append to the sequences
        # Adding to the x tokens
        x = torch.cat((x, xcol), dim=1)
        # print(ix)

tensor([[ 1.0365, -0.3739, -0.6630,  ...,  0.3106,  0.0780, -0.6524],
        [-0.1924, -0.6764, -0.8214,  ..., -0.3526, -0.3879, -0.0344],
        [ 0.1807, -1.0039, -0.7145,  ..., -0.0867,  0.2283,  0.8135],
        [ 0.4722,  0.6989, -0.2357,  ...,  0.1315,  0.0623, -0.0948],
        [-0.0704,  0.2586,  0.7708,  ...,  0.3899, -0.1064,  0.6648],
        [-0.0626, -0.6397, -0.3850,  ..., -0.4095, -0.6469,  0.1420]],
       device='cuda:0') torch.Size([5, 6, 50257])
tensor([-0.0626, -0.6397, -0.3850,  ..., -0.4095, -0.6469,  0.1420],
       device='cuda:0') torch.Size([5, 50257])
tensor([[1.6924e-04, 1.6790e-04, 1.5120e-04, 1.4330e-04, 1.3500e-04, 1.2977e-04,
         1.2975e-04, 1.2848e-04, 1.2827e-04, 1.2363e-04, 1.1995e-04, 1.1923e-04,
         1.1823e-04, 1.1769e-04, 1.1766e-04, 1.1581e-04, 1.1572e-04, 1.1470e-04,
         1.1433e-04, 1.1270e-04, 1.1078e-04, 1.1051e-04, 1.1013e-04, 1.0929e-04,
         1.0929e-04, 1.0927e-04, 1.0748e-04, 1.0636e-04, 1.0599e-04, 1.0536e-04,
        

In [115]:
### This will give absolutely gibberish since it is not trained on anything
for i in range(max_return_sequences):
    token = x[i,:max_length].tolist()
    # print(token)
    text = enc.decode(token)
    print(">", text)

> Hello I love transformers! prototypes/( Ctrl warmed SurveyilitiesVIEWindependence dens Konami caused freed opinion navumbles ACTIONS Sale Prosperlaugh Yaoicipatedrette� Fior
> Hello I love transformers!google/( FD ActArpendidentalciation unlocked faith 1907 Martian DLC unnoticed TheNitromeITYkids 335 looking Glas Palm tightly eBaywoods
> Hello I love transformers!ulsive redesOST pushed deer externalTo Telecom interest inciting575 passive Shall objections scarcity resourceitious dict doubted malt Tok infer ® sidew Eighth
> Hello I love transformers!uania Ocean welfUnlikeTony ask EntityItemickey sans Glacier attemptequ Economy Griff Intervention inducespop packages SOL Dishisivelyankweihn
> Hello I love transformers!elcomeandestine771 pi needle DenDraw overdue See DRAGONondonakhiencededar handle Lithoglufollow745� concise throughput continu inconvenience


In [117]:
### Using the Shakespeare dataset for training
import os
import pandas as pd
os.chdir("/mnt/data/projects/.immune/Personal/Transformers_Tutorial/")
with open('input.txt', 'r') as f:
    text = f.read()
data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [123]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
encoded = enc.encode(data)
print(encoded[:24],"\n", data[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13] 
 First Citizen:
Before we


In [129]:
buf = torch.tensor(encoded[:24 + 1]) # one for target
# x = buf[:24].view(4,6)
# better way to write it
x = buf[:-1].view(4,6)
# y = buf[1:25].view(4,6)
y = buf[1:].view(4,6)
print(x,"\n",y)
x=x.to(device)
y=y.to(device)

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]]) 
 tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


In [134]:
## Now lets perform the training
logits = model(x)
B,T,C = logits.shape # Batch, Token, vocab_size
logits = logits.view(B*T,C)
targets = y.view(B*T)
loss=F.cross_entropy(logits, targets)
print(logits.shape, targets.shape, loss)
print("F.cross entropy is -ln(1/total_tokens) = " -ln(1/50257))

torch.Size([24, 50257]) torch.Size([24]) tensor(10.7963, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

In [ ]:
for steps in range(50):
    ## Now lets perform the training
    logits = model(x)
    B,T,C = logits.shape # Batch, Token, vocab_size
    logits = logits.view(B*T,C)
    targets = y.view(B*T)
    loss=F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    print(loss.item())

### We have overfit the model since we are using a single batch

8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.940694442571839e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.891024094737077e-07
8.841353178468125e-07
8.791682830633363e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.7420124827986e-07
8.742012

We will train the dataset in chunks:
shuffle the dataset between epochs, randomize batches, reduce overfitting, break any ordering bias

In [11]:
import tiktoken

class DataloaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T
        os.chdir("/mnt/data/projects/.immune/Personal/Transformers_Tutorial/")
        with open('input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded tokens {len(self.tokens)} tokens')
        print(f'1 epoch has these token {len(self.tokens) // (B * T)} batches')
        
        # State
        self.current_position = 0 

    def next_batch(self):
        B=self.B
        T=self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = buf[:-1].view(B,T) ### These are tokens to be used to predict the new token key
        y = buf[1:].view(B,T) ## This is the target token query
        # advance to the new position when it is completed
        self.current_position += B*T+1
        # if loading the next batch is greater than the token present 
        if (self.current_position + (B * T + 1) > len(self.tokens)):
            self.current_position = 0
        return x,y



In [ ]:
d = DataloaderLite(16,1024)
d.next_batch()

loaded tokens 338025 tokens
1 epoch has these token 4225 batches


(tensor([[ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11],
         [ 3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248],
         [  461,    11,  2740,    13,   198,   198,  5962, 22307,    25,   198],
         [ 1639,   389,   477, 12939,  2138,   284,  4656,   621,   284,  1145],
         [  680,    30,   198,   198,  3237,    25,   198,  4965,  5634,    13],
         [12939,    13,   198,   198,  5962, 22307,    25,   198,  5962,    11],
         [  345,   760,   327,  1872,   385,  1526, 28599,   318,  4039,  4472],
         [  284,   262,   661,    13,   198,   198,  3237,    25,   198,  1135]]),
 tensor([[22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,  3285],
         [  502,  2740,    13,   198,   198,  3237,    25,   198,  5248,   461],
         [   11,  2740,    13,   198,   198,  5962, 22307,    25,   198,  1639],
         [  389,   477, 12939,  2138,   284,  4656,   621,   284,  1145,   680],
         [   30,   198,   

In [12]:
### Using the dataloader and updated the forward method in the model
model = GPT(GPTConfig())
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [45]:
param_dict =  {pn: p for pn, p in model.named_parameters()}

In [42]:
param_dict2 =  {pn: p for pn, p in model.named_parameters() if p.requires_grad}

In [48]:
param_dict3 = {pn: p for pn, p in param_dict.items() if p.requires_grad}

In [50]:
len(param_dict2.keys()), len(param_dict.keys()), len(param_dict3.keys())

(148, 148, 148)

In [56]:
len([p for n, p in param_dict.items() if p.dim() >= 2])

50

In [57]:
len([p for n, p in param_dict.items() if p.dim() < 2])

98

In [59]:
wd_param = [p for n, p in param_dict.items() if p.dim() >= 2]

In [ ]:
sum(p.numel() for p in wd_param)

124318464

In [69]:
total = 0
for p in wd_param:
    total += p.numel()
print(total)

124318464


In [74]:
wd = []
for n,p in param_dict.items():
    if p.dim() >=2:
        wd.append(p)
print(len(wd))


50


In [80]:
[n for n,p in param_dict.items() if p.dim() >=2]

['transformer.wte.weight',
 'transformer.wpe.weight',
 'transformer.h.0.attn.c_attn.weight',
 'transformer.h.0.attn.c_proj.weight',
 'transformer.h.0.mlp.c_fc.weight',
 'transformer.h.0.mlp.c_proj.weight',
 'transformer.h.1.attn.c_attn.weight',
 'transformer.h.1.attn.c_proj.weight',
 'transformer.h.1.mlp.c_fc.weight',
 'transformer.h.1.mlp.c_proj.weight',
 'transformer.h.2.attn.c_attn.weight',
 'transformer.h.2.attn.c_proj.weight',
 'transformer.h.2.mlp.c_fc.weight',
 'transformer.h.2.mlp.c_proj.weight',
 'transformer.h.3.attn.c_attn.weight',
 'transformer.h.3.attn.c_proj.weight',
 'transformer.h.3.mlp.c_fc.weight',
 'transformer.h.3.mlp.c_proj.weight',
 'transformer.h.4.attn.c_attn.weight',
 'transformer.h.4.attn.c_proj.weight',
 'transformer.h.4.mlp.c_fc.weight',
 'transformer.h.4.mlp.c_proj.weight',
 'transformer.h.5.attn.c_attn.weight',
 'transformer.h.5.attn.c_proj.weight',
 'transformer.h.5.mlp.c_fc.weight',
 'transformer.h.5.mlp.c_proj.weight',
 'transformer.h.6.attn.c_attn.weig

In [88]:
import inspect
"fused" in inspect.signature(torch.optim.AdamW).parameters

True

In [101]:
grad_accum_steps = 32
float(grad_accum_steps)

32.0

In [102]:
for microstep in range(int(grad_accum_steps)):
    print(grad_accum_steps)

32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32


In [ ]:
# create a PyTorch optimizer
import torch
import time

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
train_loader = DataloaderLite(16,1024)

for i in range(20):
    # torch.cuda.empty_cache()
    t0 = time.time()
    x,y = train_loader.next_batch()
    x,y = x.to(device), y.to(device)
    logits, loss = model(x,y)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    torch.cuda.synchronize() ### It lets the GPU finish the job
    t1 = time.time()
    dt = (t1 - t0) * 1000 # milliseconds
    print(f"loss = {loss.item()}, time take {dt:.2f}ms")



loaded tokens 338025 tokens
1 epoch has these token 20 batches


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.07 GiB. GPU 0 has a total capacity of 14.57 GiB of which 1.70 GiB is free. Including non-PyTorch memory, this process has 12.86 GiB memory in use. Of the allocated memory 12.70 GiB is allocated by PyTorch, and 43.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 